In [1]:
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import matplotlib
import keras.backend as K
matplotlib.style.use('ggplot')

## Extracting training and validation set zips

In [5]:
# Extracting train and validation
!unzip Training.zip 
!unzip Validation.zip 

Archive:  Training.zip
   creating: Training/barbie_train/
  inflating: Training/barbie_train/barbie_1.png  
  inflating: Training/barbie_train/barbie_10.png  
  inflating: Training/barbie_train/barbie_10brownian.png  
  inflating: Training/barbie_train/barbie_10cos.png  
  inflating: Training/barbie_train/barbie_10gaussian.png  
  inflating: Training/barbie_train/barbie_10quadratic.png  
  inflating: Training/barbie_train/barbie_11brownian.png  
  inflating: Training/barbie_train/barbie_11cos.png  
  inflating: Training/barbie_train/barbie_11pink.png  
  inflating: Training/barbie_train/barbie_11quadratic.png  
  inflating: Training/barbie_train/barbie_11sawtooth.png  
  inflating: Training/barbie_train/barbie_11sin.png  
  inflating: Training/barbie_train/barbie_11uniform.png  
  inflating: Training/barbie_train/barbie_12.png  
  inflating: Training/barbie_train/barbie_12brownian.png  
  inflating: Training/barbie_train/barbie_12cos.png  
  inflating: Training/barbie_train/barbie_12g

In [2]:
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = '/content/Training/'
VALID_DATA_DIR = '/content/Validation/'

In [3]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)
train_generator = datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    shuffle=True,
    target_size=IMAGE_SHAPE,
)
valid_generator = datagen.flow_from_directory(
    VALID_DATA_DIR,
    shuffle=False,
    target_size=IMAGE_SHAPE,
)

Found 548 images belonging to 2 classes.
Found 136 images belonging to 2 classes.


In [4]:
def build_model(num_classes):
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', 
                           input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model
model = build_model(num_classes=2)

In [5]:
def get_f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[get_f1]
)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 16)      2320      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 16)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 32)        4640      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 32)       0

In [18]:
EPOCHS = 40
BATCH_SIZE = 16
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE // 4,
                    epochs=EPOCHS,
                    validation_data=valid_generator,
                    validation_steps= valid_generator.samples // BATCH_SIZE // 2,
                    verbose=1
                    )

Epoch 1/40
8/8 [==============================] - 1s 175ms/step - loss: 1.3534e-04 - get_f1: 1.0000 - val_loss: 1.9501 - val_get_f1: 0.7969
Epoch 2/40
8/8 [==============================] - 1s 116ms/step - loss: 2.9308e-04 - get_f1: 1.0000 - val_loss: 1.9550 - val_get_f1: 0.7969
Epoch 3/40
8/8 [==============================] - 1s 158ms/step - loss: 2.8291e-04 - get_f1: 1.0000 - val_loss: 1.9457 - val_get_f1: 0.8047
Epoch 4/40
8/8 [==============================] - 1s 113ms/step - loss: 1.8067e-04 - get_f1: 1.0000 - val_loss: 1.9474 - val_get_f1: 0.8125
Epoch 5/40
8/8 [==============================] - 1s 113ms/step - loss: 2.2052e-04 - get_f1: 1.0000 - val_loss: 1.9379 - val_get_f1: 0.8125
Epoch 6/40
8/8 [==============================] - 1s 113ms/step - loss: 2.1578e-04 - get_f1: 1.0000 - val_loss: 1.9460 - val_get_f1: 0.8125
Epoch 7/40
8/8 [==============================] - 1s 162ms/step - loss: 2.9579e-04 - get_f1: 1.0000 - val_loss: 1.9534 - val_get_f1: 0.8125
Epoch 8/40
8/8 [====

## Evaluating on the validation set

In [19]:
model.evaluate(valid_generator)

5/5 [==============================] - 0s 52ms/step - loss: 2.1019 - get_f1: 0.8250


[2.1018967628479004, 0.8249999284744263]

In [20]:
model.save_weights('Jayveersinh_Raj')